In [ ]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import pandas as pd
import numpy as np
import pickle
import math

In [ ]:
seed = 99

# Build/Load Model

In [ ]:
batch1 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch1.dat", 
                    header=None, sep=' \d+:')

batch2 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch2.dat", 
                    header=None, sep=' \d+:')

batch3 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch3.dat", 
                    header=None, sep=' \d+:')

batch4 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch4.dat", 
                    header=None, sep=' \d+:')

batch5 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch5.dat", 
                    header=None, sep=' \d+:')

batch6 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch6.dat", 
                    header=None, sep=' \d+:')

batch7 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch7.dat", 
                    header=None, sep=' \d+:')

batch8 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch8.dat", 
                    header=None, sep=' \d+:')

batch9 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch9.dat", 
                    header=None, sep=' \d+:')

batch10 = pd.read_csv("/home/admin2/ML_NIC/datasets/gas+sensor+array+drift+dataset/batch10.dat", 
                    header=None, sep=' \d+:')

In [ ]:
dataset = pd.concat([batch1, batch2, batch3, batch4, batch5,
                    batch6, batch7, batch8, batch9, batch10], ignore_index=True, axis=0)

In [ ]:
features = dataset.drop(columns=[0])
labels = dataset[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=seed,
                                                   stratify=labels)

In [ ]:
minmax = MinMaxScaler(feature_range=(0, 1), clip=True)

In [ ]:
X_train = minmax.fit_transform(X_train)
X_test = minmax.transform(X_test)

In [ ]:
# Load up model
with open("model.pkl", 'rb') as f:
    clf = pickle.load(f)

# Micro-C Model

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
values = clf.tree_.value

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True

print(
    "The binary tree structure has {n} nodes and has "
    "the following tree structure:\n".format(n=n_nodes)
)
for i in range(n_nodes):
    if is_leaves[i]:
        print(
            "{space}node={node} is a leaf node with value={value}.".format(
                space=node_depth[i] * "\t", node=i, value=values[i]
            )
        )
    else:
        print(
            "{space}node={node} is a split node with value={value}: "
            "go to node {left} if X[:, {feature}] <= {threshold} "
            "else to node {right}.".format(
                space=node_depth[i] * "\t",
                node=i,
                left=children_left[i],
                feature=feature[i],
                threshold=threshold[i],
                right=children_right[i],
                value=values[i],
            )
        )

In [ ]:
# Decide number of bits required for the path storage
num_features = clf.n_features_in_
classes = clf.classes_
num_leaves = int(np.sum(is_leaves))
num_result_words = math.ceil(num_leaves/32.0)
result_init = hex((2**(num_leaves % 33))-1)
features_per_result_core = int(32.0/num_result_words)
num_result_cores = [math.ceil(num_features / features_per_result_core)]

while(num_result_cores[-1] != 1):
    num_result_cores.append(math.ceil(num_result_cores[-1] / features_per_result_core))

In [ ]:
total_bits = 16
dec_bits = 13

def num_translate(number, total_bits, dec_bits):
    return round(number * (2**dec_bits))

In [ ]:
print("__lmem uint32_t path_class[] = {", end='')
for i in range(is_leaves.shape[0]):
    if (is_leaves[i]):
        print(f"{classes[np.argmax(values[i][0])]}", end=', ')
print('};')

In [ ]:
# Depth-first search through nodes
def depth_first(root, left, right, prediction, feature, threshold, output, feature_id, num_result_words, feat_file):
    if (left[root] == right[root]):  # leaf node is base case
        if (output == ''):
            return
        
        if feat_file:
            path_id = np.sum(is_leaves[:root])
            output = output.lstrip("&& ")
            output = output.rstrip(" ")

            if (num_result_words > 1):
                result_index = num_result_words - int(path_id/32) - 1
                path_id = path_id % 32
                feat_file.write(f"        if (!({output}))\n            result_gpr[{result_index}] &= ~(1 << {path_id});\n")
            else:
                feat_file.write(f"        if (!({output}))\n            result_gpr &= ~(1 << {path_id});\n")
            feat_file.write("\n")
            
        else:
            path_id = np.sum(is_leaves[:root])
            output = output.lstrip("and ")
            output = output.rstrip(" ")

            if (num_result_words > 1):
                result_index = num_result_words - int(path_id/32) - 1
                path_id = path_id % 32
                print(f"    if (not({output})):\n        result_gpr[{result_index}] &= ~(1 << {path_id})")
            else:
                print(f"    if (not({output})):\n        result_gpr &= ~(1 << {path_id})")
            print("")
            
    else:
        
        if feat_file:
            net_threshold = num_translate(threshold[root], total_bits, dec_bits)
            if (feature[root] == feature_id):
                left_output = output + f"&& feature <= {net_threshold} "
                right_output = output + f"&& feature > {net_threshold} "
            else:
                left_output = output
                right_output = output

            depth_first(left[root], left, right, prediction, feature, threshold, 
                        left_output, feature_id, num_result_words, feat_file)
            depth_first(right[root], left, right, prediction, feature, threshold, 
                        right_output, feature_id, num_result_words, feat_file)
            
        else:
            net_threshold = num_translate(threshold[root], total_bits, dec_bits)
            if (feature[root] == feature_id):
                left_output = output + f"and feature <= {net_threshold} "
                right_output = output + f"and feature > {net_threshold} "
            else:
                left_output = output
                right_output = output

            depth_first(left[root], left, right, prediction, feature, threshold, 
                        left_output, feature_id, num_result_words, feat_file)
            depth_first(right[root], left, right, prediction, feature, threshold, 
                        right_output, feature_id, num_result_words, feat_file)

        

In [ ]:
for feat_id in range(num_features):    
    print(f"def feature{feat_id+1}(feature):")
    
    if (num_result_words > 1):
        print("    result_gpr = [0xffffffff", end='')
        for i in range(1, num_result_words):
            print(", 0xffffffff", end='')
        print(']')
    else:
        print(f"    result_gpr = {result_init}")
        
    depth_first(0, children_left, children_right, values, feature, threshold, "", feat_id, num_result_words, None)
    print("    return result_gpr")
    print()
    print()

# Test Micro-C Model (Python Version)

In [ ]:
def feature1(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 373)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 373)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 373)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 373)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 200)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 652)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 652)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 514)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature <= 514)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 514)):
        result_gpr[7] &= ~(1 << 12)

    return result_gpr


def feature2(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 14)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 14)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 14)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 14)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 14)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 14)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 14)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 14)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature <= 14)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 14)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 14)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 14)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 14)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 14)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature > 14)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 14)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 14)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 14)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 14)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 14)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature > 14)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 14)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 8)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 8)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 8)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 8)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 8)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 15)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 15)):
        result_gpr[0] &= ~(1 << 29)

    return result_gpr


def feature3(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 63)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 63)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 63)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 4)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 4)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 4)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 189)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 189)):
        result_gpr[4] &= ~(1 << 13)

    return result_gpr


def feature4(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 129)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature > 129)):
        result_gpr[7] &= ~(1 << 27)

    return result_gpr


def feature5(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 17)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 17)):
        result_gpr[0] &= ~(1 << 1)

    return result_gpr


def feature6(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7717)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 7717)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 7717)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 3632)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 3632)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 3632)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 8192)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 8192)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 8192)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 5631)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 5631 and feature <= 7855)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 5631 and feature <= 7855)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 5631 and feature <= 7855)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 5631 and feature <= 7855)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 5631 and feature <= 7855)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 5631 and feature <= 7855)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 5631 and feature > 7855)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 5631 and feature > 7855)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 8023)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 8023)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 8112)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 8112)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 8112)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 8112)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 8112)):
        result_gpr[6] &= ~(1 << 31)

    return result_gpr


def feature7(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7234)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 7234)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 7234)):
        result_gpr[6] &= ~(1 << 13)

    return result_gpr


def feature8(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7637)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 7637)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 7637)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 7597)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 7597)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 7597)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 7597)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 7597)):
        result_gpr[5] &= ~(1 << 0)

    return result_gpr


def feature9(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1432)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature <= 1432)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature <= 1432)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 1432)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature <= 1432)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 1432)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 1432)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 1432)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature > 1432)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 1432)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 1432)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 1432 and feature <= 1516)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 1432 and feature > 1516)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 1432)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 1432)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 1432)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 1432)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 1432)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 1432)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 1432)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 1432)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 283)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 283)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 283)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 283)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 698)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 698)):
        result_gpr[6] &= ~(1 << 28)

    return result_gpr


def feature10(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 9)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 9)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 16)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature <= 16)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature <= 16)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature > 16)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 16)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 16)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 16)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 16)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature <= 63)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 63)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 63)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 63)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 63)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 60)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 60)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 60)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 60)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 60)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 60)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 3)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 3)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 3)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 3)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 3)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 3)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 3)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 3)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 3)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 3)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 3)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 3)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 3)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 18)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 18)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 18)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 18)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 92)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 92)):
        result_gpr[0] &= ~(1 << 20)

    return result_gpr


def feature11(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 103)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 103)):
        result_gpr[3] &= ~(1 << 22)

    return result_gpr


def feature12(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3513)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 3513)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 3513)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 757 and feature <= 443)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 757 and feature > 443 and feature <= 452)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 757 and feature > 443 and feature <= 452)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 757 and feature > 443 and feature > 452)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 757 and feature > 443)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 757 and feature > 443)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 757 and feature > 443)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 757 and feature > 443)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 757 and feature > 443)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature <= 757 and feature > 443)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 757)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature <= 757)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 757)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 757)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 757)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 757)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 757)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 757)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 757)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 757)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 757)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 757)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 757)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 757)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 757)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 757)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 757)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 757)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 757)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 757)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 757 and feature <= 831)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 757 and feature > 831)):
        result_gpr[3] &= ~(1 << 13)

    return result_gpr


def feature13(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 778)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 778)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 778)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 778)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 778)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 778)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 778)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 778)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 778)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 778)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 778)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 778)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 778)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 778)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 778)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 778)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 778)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 778)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 778)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 778)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 778)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 778)):
        result_gpr[5] &= ~(1 << 24)

    return result_gpr


def feature14(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 6643)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 6643)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 6643)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 6643)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 6643)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 6643)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 6643)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 6643)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 7769)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 7769)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 7769)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 7769)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 7769)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 7769)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 7769)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 7769)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 7769)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 7769)):
        result_gpr[1] &= ~(1 << 27)

    return result_gpr


def feature15(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7777)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 7777)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 7777)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 7777)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 7777)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 7244)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 7244)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 7244)):
        result_gpr[5] &= ~(1 << 5)

    return result_gpr


def feature16(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7708)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 7708)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 7708)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 7708)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 7657)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature > 7657)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 7684)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 7684)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 7378)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature <= 7378)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature <= 7378)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 7378)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature <= 7378)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 7378)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 7378)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 7378)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature <= 7378)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 7378)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 7378)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 7378)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 7378)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature <= 7378)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature <= 7378)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 7378)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 7378)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature <= 7378)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature <= 7378)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 7378 and feature <= 7477)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 7378 and feature > 7477)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 7378)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature > 7378)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 7378)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 7378)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 7378)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 7378)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature17(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2653)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 2653)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 2653 and feature <= 2484)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 2653 and feature > 2484)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 2653)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 2179)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature <= 2179)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 2179)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 2179)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature > 2179)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 2179)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 2179)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 2179)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 2179)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 2179)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 2179)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 2179)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 2179)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 2179)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 2179)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 2179)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 2179)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 2179)):
        result_gpr[6] &= ~(1 << 17)

    return result_gpr


def feature18(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 147)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 147)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature > 147)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature > 147)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature > 147)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 147)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 992)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature <= 992)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 992)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 992)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 1799)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 1799)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 1799)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 1799)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 1799)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 1799)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 1799)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 680)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 680)):
        result_gpr[3] &= ~(1 << 4)

    return result_gpr


def feature19(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1042)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 1042)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 1042)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature20(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 38)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 38)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 38)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 64)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature <= 64)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 64)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 64)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 202)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 202)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature <= 202)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 202)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 202)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 202)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 202)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 202)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 202)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 202)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 202)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 202)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 202)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 202)):
        result_gpr[2] &= ~(1 << 8)

    return result_gpr


def feature21(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature22(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature23(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7757)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 7757)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 7757)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 7727)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 7727)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 7756)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 7756)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 7756)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 7756)):
        result_gpr[4] &= ~(1 << 9)

    return result_gpr


def feature24(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature25(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2844)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 2844)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 2681)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 2681)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 2681)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 2681)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 2681)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 2681)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 2681)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 2681)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 2681)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 2681)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 2681)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 2681)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 2681)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 2681)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 2681)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 2681)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 2681)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 2681)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 2681)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 2681)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 2681)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature <= 2681)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 2681)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 2681)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 2681)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 2681)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 2681)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 2681)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 2681)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 2681)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 3390)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 3390)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 4018)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 4018)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature <= 4018)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 4018)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature <= 4018)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4018)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 4018)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 4018)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 4018)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 4018)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 4018)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 4018)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 4018)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 4018)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 4018)):
        result_gpr[7] &= ~(1 << 3)

    return result_gpr


def feature26(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 261)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature <= 261)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 261)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 261)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 261)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 302)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 302)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 302)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 302)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 302)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 302)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 302)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 302)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 302)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 302)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 302)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 302)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 302)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 302)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 302)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 302)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 302)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 302)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 302)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature <= 302)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 302)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 302)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 302)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 302)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 302)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 302)):
        result_gpr[6] &= ~(1 << 31)

    return result_gpr


def feature27(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature28(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 519)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 519)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature <= 519)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 519)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature <= 519)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 519)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 519)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 519)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 519)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 519)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 519)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature <= 519)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 519)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature <= 519)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature <= 519)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 519)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 519)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 519)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 519)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 519)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature <= 519)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature <= 519)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature <= 519)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature <= 519)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 519)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 519)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 519)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 519)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 519)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature29(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2251)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 2251)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 33)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 33)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 38)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 38)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 38)):
        result_gpr[0] &= ~(1 << 0)

    return result_gpr


def feature30(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature31(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7964)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 7964)):
        result_gpr[1] &= ~(1 << 7)

    return result_gpr


def feature32(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 8140)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature > 8140)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 8171)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 8171)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 8171)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 8171)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 8171)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 8136)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 8136)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 8136)):
        result_gpr[2] &= ~(1 << 8)

    return result_gpr


def feature33(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1065)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 1065)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 1065)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 1065)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 1065)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 1065)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 1065)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 1065)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature > 1065)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 1065)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 1065)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 1065)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 2113)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 2113)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 3298)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 3298)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 846)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 846)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 846)):
        result_gpr[7] &= ~(1 << 17)

    return result_gpr


def feature34(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 29)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 29)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 29)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature > 29)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 82)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 82)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 82)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 82)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 82)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 82)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 82)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 82)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 82)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 82)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 82)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 82)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 82)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 82)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 82)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 82 and feature <= 36)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 82 and feature > 36)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 82)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 82)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 82)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 82)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 82)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 82)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 82)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 82)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 82)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature <= 82)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 82)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 82)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 82)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 57)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 57)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 57)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 57)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 57)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 57)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 57)):
        result_gpr[6] &= ~(1 << 28)

    return result_gpr


def feature35(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 312)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 312)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 312)):
        result_gpr[5] &= ~(1 << 17)

    return result_gpr


def feature36(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 94)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature <= 94)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 94)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 94)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 94)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 94)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 94)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 94)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 94)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 94)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 94)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 94)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 94)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 94)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 94)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature <= 94)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature <= 94)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature <= 94)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 94 and feature <= 138)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 94 and feature <= 138)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 94 and feature > 138)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 94 and feature > 138)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 94 and feature > 138)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 94 and feature > 138)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 94 and feature > 138)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 94 and feature > 138)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 94 and feature > 138)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 94 and feature > 138)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 94 and feature > 138)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 94 and feature > 138)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 94 and feature > 138)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 94 and feature > 138)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 94 and feature > 138)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 94 and feature > 138)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 94 and feature > 138)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 94 and feature > 138)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 94 and feature > 138)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 94 and feature > 138)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 94 and feature > 138)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 94 and feature > 138)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 94 and feature > 138)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 94 and feature > 138)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 94)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 94)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 94)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 94)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 94)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 94)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 94 and feature <= 280)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 94 and feature <= 280)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 94 and feature <= 280)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 94 and feature > 280)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature > 94)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 94)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 94)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 94 and feature <= 119)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 94 and feature > 119)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 94 and feature > 119)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 94 and feature > 119)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 94 and feature > 119)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 94 and feature > 119)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 94 and feature > 119)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 94 and feature > 119)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 94)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 94)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 94)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 94 and feature <= 239)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 94 and feature > 239)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 94 and feature > 239)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 94)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 94)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 94)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 94)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 94)):
        result_gpr[6] &= ~(1 << 31)

    return result_gpr


def feature37(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 15)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature > 15)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 75)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 75)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 75)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 75)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 75)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature38(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature39(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7735)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 7735)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 7735)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 7635)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 7635)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature <= 7635)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 7635)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 7635)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 7635)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 7635)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 7635)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 7635)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 7635)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 7349)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 7349)):
        result_gpr[7] &= ~(1 << 14)

    return result_gpr


def feature40(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 8122)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 8122)):
        result_gpr[4] &= ~(1 << 28)

    return result_gpr


def feature41(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature42(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 1)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 1)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 1)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 1)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 1)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 1)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 0)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 0)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 0)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 0)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 0)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 0)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 0)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 0)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 0)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 0)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 0)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 0)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 0)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 0)):
        result_gpr[2] &= ~(1 << 13)

    return result_gpr


def feature43(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 203)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 203)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 203)):
        result_gpr[3] &= ~(1 << 4)

    return result_gpr


def feature44(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature45(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 132)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 132)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 132)):
        result_gpr[0] &= ~(1 << 1)

    return result_gpr


def feature46(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 6499)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 6499)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 5009)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 5009)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 5009)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 5009)):
        result_gpr[4] &= ~(1 << 15)

    return result_gpr


def feature47(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature48(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 8072)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 8072)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 8030)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 8030)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 8030)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 8030)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 8030)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 8030)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 8030)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 8030)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 8030)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 8030)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 8030)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 8030)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature49(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2281)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature > 2281)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 2281)):
        result_gpr[6] &= ~(1 << 8)

    return result_gpr


def feature50(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 2)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 2)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 2 and feature <= 2)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 2 and feature > 2)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 2 and feature > 2)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 2)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 5)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 5)):
        result_gpr[3] &= ~(1 << 26)

    return result_gpr


def feature51(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 518)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 518)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 518)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 752)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 752)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 752)):
        result_gpr[6] &= ~(1 << 28)

    return result_gpr


def feature52(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature53(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature54(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature55(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 5687)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 5687)):
        result_gpr[1] &= ~(1 << 13)

    return result_gpr


def feature56(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7893)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 7893)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 7893)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 7893)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 7893)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 7893)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 7893)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 7893)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 7893)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 7893)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 7893)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 7893)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 7893)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 7893)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 7893)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 7893)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 7893)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 7893)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 7893)):
        result_gpr[5] &= ~(1 << 9)

    return result_gpr


def feature57(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2136)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 2136)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 2136)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 2136)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 2136)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 2136)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 2136)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 2136)):
        result_gpr[3] &= ~(1 << 24)

    return result_gpr


def feature58(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 1)):
        result_gpr[3] &= ~(1 << 30)

    return result_gpr


def feature59(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 835)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 835)):
        result_gpr[3] &= ~(1 << 15)

    return result_gpr


def feature60(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 237)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 237)):
        result_gpr[2] &= ~(1 << 13)

    return result_gpr


def feature61(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 277)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 277)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 1400)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 1400)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 1400)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature62(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7986)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 7986)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 7315)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 7315)):
        result_gpr[0] &= ~(1 << 17)

    return result_gpr


def feature63(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature64(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature65(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 79)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature > 79)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 79)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 79)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature > 79)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 79)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 79)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 79)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 79)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 79)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 79)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 79)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 79)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 79)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 79)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 165)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 165)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 165)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 165)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature > 165)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 165)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 165)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 165)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 165)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 165)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 165)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 2372)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 2372)):
        result_gpr[2] &= ~(1 << 2)

    return result_gpr


def feature66(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 136)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 136)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 136)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature > 136)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 136)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 136)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 97)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 97)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 97)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 97)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 97)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 97)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 566)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 566)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 310)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 310)):
        result_gpr[1] &= ~(1 << 9)

    return result_gpr


def feature67(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 180)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 180)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 180)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 180)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 180)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 180)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 180)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 180)):
        result_gpr[4] &= ~(1 << 20)

    return result_gpr


def feature68(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 141)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 141)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 141)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 141)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 141)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 141)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 141)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 141)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 141)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 141)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 141)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 141)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 141)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 141)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 141)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 141)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 141)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 141)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 141)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 141)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 141)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 141)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature > 141)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 141)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 141)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 141)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 141)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 141)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 141)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 141)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 141)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 141)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 141)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 141)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 141)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature > 141)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 141)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 1313)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 1313)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 1313)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 1313)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 1313)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 1313)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 1313)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 1313)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 1313)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 1313)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 1313)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature <= 1313)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 1313)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 1313)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 621)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 621)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 621)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 547)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 547)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 547)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 547)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 547)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 547)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 547)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 547)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 547)):
        result_gpr[5] &= ~(1 << 21)

    return result_gpr


def feature69(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 200)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 200)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 200)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 200)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 200)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 200)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 200)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 200)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 200)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 200)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 200)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 200)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 200)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 200)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 200)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 200)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature <= 200)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 200)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 200)):
        result_gpr[7] &= ~(1 << 0)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 200 and feature <= 404)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 200 and feature > 404)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 200 and feature > 404)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 200 and feature > 404)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 200 and feature <= 729)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 200 and feature > 729)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 200 and feature > 729)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 200 and feature <= 538)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 200 and feature > 538)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 200)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature > 200)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 200)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 200)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 200)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 200)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature70(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 8021)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 8021)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 8021)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 8021)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature71(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7653)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 7653)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 7653)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 7653)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 7739)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 7739)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature <= 7739)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 7739)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature <= 7739)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 7739)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 7739)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 7739)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 7739)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 7739)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 7739)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature <= 7739)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 7739)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature <= 7739)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature <= 7739)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 7739)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 7739)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 7739)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 7739)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 7739)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature <= 7739)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 7739)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 7739)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 7739)):
        result_gpr[7] &= ~(1 << 12)

    return result_gpr


def feature72(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7477)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 7477)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 7477)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 7477)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 7477)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 7477)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 7477)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 7477)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 7477)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 7477)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 7477)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 7477)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 7477)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 7613)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 7613)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 7613)):
        result_gpr[6] &= ~(1 << 20)

    return result_gpr


def feature73(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 223)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 223)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 223)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 223)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 223)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 223)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 223)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 223)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 223)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 223)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 486)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 486)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 486)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 486)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 860)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 860)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 383)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 383)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 383)):
        result_gpr[1] &= ~(1 << 7)

    return result_gpr


def feature74(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 384)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 384)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 384)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 384)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 384)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 384)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 384)):
        result_gpr[3] &= ~(1 << 5)

    return result_gpr


def feature75(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 194)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 194)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature > 194)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 194)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 194)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 3988)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 3988)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 3988)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 3988)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 3988)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 3988)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 53)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 53)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 53)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 541)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 541)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 541)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 541)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 541)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 541)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 541)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 541)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 541)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 541)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 541)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 541)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 541)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 541)):
        result_gpr[3] &= ~(1 << 13)

    return result_gpr


def feature76(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 633)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 633)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 304)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature <= 304)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 304)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 304)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 304)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 304)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 304)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 304)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 304)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 304)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 304)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 304)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 304)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 304)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 304)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 304)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 304)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 304)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 304)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 304)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 304)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 304)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 304)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 304)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 304)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 304)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 304)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 304)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 304)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 304)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 304)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 304)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 359)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 359)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 359)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 359)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 359)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 359)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 359)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 359)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 359)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 757)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 757)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 757)):
        result_gpr[6] &= ~(1 << 31)

    return result_gpr


def feature77(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 201)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 201)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 685)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 685)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 685 and feature <= 452)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 685 and feature > 452)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 685)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 685)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 685)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 685)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 685)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 685)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 685)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 685)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 685)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 685)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 685)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 685)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 685)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 685)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 685)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 685)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 685)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 685)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature > 685)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 685)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 685)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 685)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 685)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 685)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 685)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 685)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 685)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 685)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 685)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 685)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 685)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 685)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 685)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 685)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 685)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 685)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 178)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 178)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 178)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 178)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 178)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 178)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 178)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 178)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 178)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 1042)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 1042)):
        result_gpr[6] &= ~(1 << 0)

    return result_gpr


def feature78(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7812)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 7812 and feature <= 7461)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 7812 and feature > 7461)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 7812 and feature > 7461)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 7812)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 7812)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 7812)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 7812)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 7812)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 7812)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 7812)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 7812)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 7812)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 7812)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 7812)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 7812)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 7812)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 7812)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 7812)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature <= 7812)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 7812)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 7812)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 7812)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 7812)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 7812)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 7812)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 7812)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature <= 7812)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature <= 7812)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature <= 7812)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 7812)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 7812)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 7812)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 7812)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature > 7812)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 7812)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 7812)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 7812)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 7812)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 7812)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 7812)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 7812)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 7812)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 7812)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 7812 and feature <= 7919)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 7812 and feature <= 7919)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 7812 and feature <= 7919)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 7812 and feature > 7919)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 7812 and feature > 7919)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 7812 and feature > 7919)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 7812)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 7812)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 7812)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 7812)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 7812)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 7812)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 7812)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 7812)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 7812)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 7812)):
        result_gpr[6] &= ~(1 << 31)

    return result_gpr


def feature79(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7396)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 7396)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 7396)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 7396)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 7396)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 7396)):
        result_gpr[3] &= ~(1 << 6)

    return result_gpr


def feature80(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature81(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1343)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 1343)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 1343)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 1343)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 1749)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 1749)):
        result_gpr[6] &= ~(1 << 24)

    return result_gpr


def feature82(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 264)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 264)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 264)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 264)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 264)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 264)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 264)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 264)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 264)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 264)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 264)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 264)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 264)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature > 264)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 264)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 264)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature > 264)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 264)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 264)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 264)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 264)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 264)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 264)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 264)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 264)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 264)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 264)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature > 264)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 296)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature <= 296)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 296)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 296)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 296)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 296)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 296)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 296)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 296)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature > 296)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 296)):
        result_gpr[7] &= ~(1 << 11)

    return result_gpr


def feature83(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1128)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 1128)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 208)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 208)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 208)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 208)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 208)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 208)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 208)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 208)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 208)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 208)):
        result_gpr[2] &= ~(1 << 13)

    return result_gpr


def feature84(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1430)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 1430)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 1430)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 1430)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 230)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 230)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 230)):
        result_gpr[1] &= ~(1 << 14)

    return result_gpr


def feature85(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 37)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature > 37)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature > 37)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 404)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 404)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 404)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 434)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 434)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 434)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 434)):
        result_gpr[5] &= ~(1 << 18)

    return result_gpr


def feature86(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature87(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7125)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 7125)):
        result_gpr[0] &= ~(1 << 26)

    return result_gpr


def feature88(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature89(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1343)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 1343)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 1343)):
        result_gpr[1] &= ~(1 << 9)

    return result_gpr


def feature90(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1485)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 1485)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 1485)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 1485)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 1485)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 1485)):
        result_gpr[0] &= ~(1 << 4)

    return result_gpr


def feature91(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 230)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature > 230)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 5230)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 5230)):
        result_gpr[2] &= ~(1 << 4)

    return result_gpr


def feature92(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 97)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 97)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 97)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature > 97)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 97)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 97)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 97)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 97)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 97)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 97)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 97)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 97)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 97)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 97)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 2441)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 2441)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature <= 2441)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 2441)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 2441)):
        result_gpr[7] &= ~(1 << 3)

    return result_gpr


def feature93(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature94(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature95(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7217)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 7217)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 7217)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 6451)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 6451)):
        result_gpr[4] &= ~(1 << 15)

    return result_gpr


def feature96(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7929)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 7929)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 7990)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 7990)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 7807)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 7807)):
        result_gpr[5] &= ~(1 << 24)

    return result_gpr


def feature97(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1393)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 1393)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 3912)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 3912)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 3912)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 3912)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 3912)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 3912)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 3912)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 3912)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 3912)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 3912)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 3912)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 3912)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 3912)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 3912)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 3912)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 3912)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 3912)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 3912)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 3912)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 3912)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 4869)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 4869)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 4869)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 4869)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 4869)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature98(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1574)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 1574)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 501)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 501)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 501)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 501)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 501)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 501)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 501)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 501)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 501)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 501)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature99(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 283)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 283)):
        result_gpr[2] &= ~(1 << 6)

    return result_gpr


def feature100(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 464)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 464)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 464)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 756)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 756)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 756)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 756)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 756)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 764)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 764)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 764)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 764)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 764)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 764)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 764)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature101(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 349)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 349)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 349)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 349)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 349)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 349)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 349)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 349)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 349 and feature <= 466)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 349 and feature > 466)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature > 349 and feature > 466)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 349 and feature > 466)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 349 and feature > 466)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 349 and feature > 466)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 349 and feature > 466)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 349 and feature > 466)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 349 and feature > 466)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 349)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 349)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 349)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 349)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 349)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 349)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 349)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 349)):
        result_gpr[4] &= ~(1 << 2)

    return result_gpr


def feature102(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7936)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 7936)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 7580)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 7580)):
        result_gpr[1] &= ~(1 << 16)

    return result_gpr


def feature103(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7644)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 7644)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 7600)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 7600)):
        result_gpr[3] &= ~(1 << 28)

    return result_gpr


def feature104(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 8069)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 8069)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 8069)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 8069)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 8069)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 8069)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 8069)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 8069)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 8069)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 8069)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 8069)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 7995)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 7995)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 8070)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 8070)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 8127)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 8127)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 8103)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 8103)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 8103)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 8103)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 8103)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 8103)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 8103)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 8103)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 8103)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 8103)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 8103)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 8103)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 8103)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 8103)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 8103)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 8103)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 8103)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 8103)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 8103)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 8103)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 8103)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 8103)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 8103)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 8103)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 8103)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 8103)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 8103)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 8103)):
        result_gpr[3] &= ~(1 << 1)

    return result_gpr


def feature105(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 2098)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 2098)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 2098)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 2098)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 2098)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 2098)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 2098)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 2098)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 2098)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 2098)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 2098)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 2098)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 2098)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 2098)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature > 2098 and feature <= 2298)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature > 2098 and feature <= 2298)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 2098 and feature <= 2298)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 2098 and feature > 2298)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature > 2098 and feature > 2298)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature > 2098)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature > 2098 and feature <= 3204)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature > 2098 and feature > 3204)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 2098 and feature > 3204)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature <= 1468)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 1468)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 1468)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 1468)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 1468)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 1468)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 1468)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 1468)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 1468)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 1468)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 1468)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 1468)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 1468)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 1468)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 1468)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 1468)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 1468)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 1468)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 1468)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 1468)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 1468)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 1468)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 1468)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 1468)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 1468)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 1468)):
        result_gpr[5] &= ~(1 << 24)

    return result_gpr


def feature106(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2076)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 2076)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 2076)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 2076)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 2076)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 2076)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 2076)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 2076)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 2076)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 2076)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 2076)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 2076 and feature <= 708)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 2076 and feature > 708)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 2076)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 2076)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature > 2076)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 4404)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature <= 4404)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 4404)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature > 4404)):
        result_gpr[7] &= ~(1 << 0)

    if (not(feature <= 4107)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 4107)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 991)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 991)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature <= 2656)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 2656)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 2656)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 2656)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 1752)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature > 1752)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 1752)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 1752)):
        result_gpr[1] &= ~(1 << 7)

    return result_gpr


def feature107(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature108(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1325)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 1325)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 1325)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 1325 and feature <= 1170)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 1325 and feature > 1170)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 1325 and feature > 1170)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 1325 and feature > 1170)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 1325)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature <= 1325)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature <= 1325)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 1325)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 1325)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 1325)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 1325)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 723)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 723)):
        result_gpr[2] &= ~(1 << 27)

    return result_gpr


def feature109(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 24)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 24)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 24)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 24)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 24)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 24)):
        result_gpr[4] &= ~(1 << 9)

    return result_gpr


def feature110(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature111(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7710)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 7710)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 7710)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 7636)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 7636)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 7636)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 7636)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 7636)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 7636)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 7636)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 7636)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 7636)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 7636)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 7636)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 7636)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 7636)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 7636)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 7636)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 7636)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 7636)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 7636)):
        result_gpr[5] &= ~(1 << 31)

    return result_gpr


def feature112(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 8017)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 8017)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 7978)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 7978)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 7978)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 7978)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 7978)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 7978)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 7978)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 7978)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 7978)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 7978)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 7988)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 7988)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 7988)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 7988)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 7988)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 7988)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 7988)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 7988)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 7988)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature113(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1875)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 1875)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 1875)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 1875)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 1875)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 1875)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature > 1875)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature > 1875)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 1875)):
        result_gpr[4] &= ~(1 << 28)

    return result_gpr


def feature114(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 81)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 81)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 81)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature > 81)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 1733)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 1733)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 1733)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 1733)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 1733)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature > 1733)):
        result_gpr[7] &= ~(1 << 16)

    return result_gpr


def feature115(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2902)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 2902)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 1118)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 1118)):
        result_gpr[4] &= ~(1 << 1)

    return result_gpr


def feature116(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 1750)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 1750)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 1750)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 1750)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 1750)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 1750)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 1750)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 1750)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 1750)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 1750)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 1750)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 1750)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 1750)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 1750)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 1750)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature <= 1750)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 1750)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature > 1750)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature > 1750)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 1750)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature > 1750)):
        result_gpr[7] &= ~(1 << 0)

    return result_gpr


def feature117(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 181)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 181)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 702)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 702)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 597)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 597)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 597)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 597)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 597)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 597)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 597)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 597)):
        result_gpr[0] &= ~(1 << 18)

    return result_gpr


def feature118(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7909)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 7909)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature <= 7909)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 7909)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 7909)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 7199)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 7199)):
        result_gpr[1] &= ~(1 << 2)

    return result_gpr


def feature119(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 5816)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 5816)):
        result_gpr[2] &= ~(1 << 8)

    return result_gpr


def feature120(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7872)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 7872)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 7872)):
        result_gpr[4] &= ~(1 << 8)

    return result_gpr


def feature121(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1501)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature > 1501)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 1501)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 3005)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 3005)):
        result_gpr[3] &= ~(1 << 20)

    return result_gpr


def feature122(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 118)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 118)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 118)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 118)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 118)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 118)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature > 118)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature > 118)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 379)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 379)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 855)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 855)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 855)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 855)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 855 and feature <= 811)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 855 and feature > 811)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 855 and feature > 811)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 855)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 855)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 855)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 855)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 855)):
        result_gpr[6] &= ~(1 << 24)

    return result_gpr


def feature123(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature124(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 34)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 34)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 172)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature <= 172)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature <= 172)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 172)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 172)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 172)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 172)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 172)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 172)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 172)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 172)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 172)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 172)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 172)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 172)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 172)):
        result_gpr[6] &= ~(1 << 30)

    if (not(feature > 172)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 172)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 285)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 285)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 285)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 285)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 285)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 285)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 285)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 285)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 285)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 285)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 285)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 285)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 285)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 285)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 285)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 285)):
        result_gpr[0] &= ~(1 << 12)

    return result_gpr


def feature125(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 625)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 625)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 625)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 625)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 625)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 625)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 625)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 625)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 625)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 625 and feature <= 978)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 625 and feature > 978)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 1030)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 1030)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 1030)):
        result_gpr[0] &= ~(1 << 18)

    return result_gpr


def feature126(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7848)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 7848)):
        result_gpr[6] &= ~(1 << 5)

    return result_gpr


def feature127(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    return result_gpr


def feature128(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 7906)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 7906)):
        result_gpr[2] &= ~(1 << 19)

    return result_gpr



In [ ]:
def dtree_aggr(result1, result2, result3, result4, result5, result6, result7, result8, result9, 
                         result10, result11, result12, result13, result14, result15, result16, result17, 
                         result18, result19, result20, result23, result25, result26, result28, result29, 
                         result31, result32, result33, result34, result35, result36, result37, result39,
                         result40, result42, result43, result45, result46, result48, result49, result50, 
                         result51, result55, result56, result57, result58, result59, result60, result61,  
                         result62, result65, result66, result67, result68, result69, result70, result71, 
                         result72, result73, result74, result75, result76, result77, result78, result79,
                         result81, result82, result83, result84, result85, result87, result89, result90, 
                         result91, result92, result95, result96, result97, result98, result99, result100, 
                         result101, result102, result103, result104, result105, result106, result108, 
                         result109, result111, result112, result113, result114, result115, result116, 
                         result117, result118, result119, result120, result121, result122, result124, 
                         result125, result126, result128):
    
    path_class = [3, 2, 2, 2, 6, 3, 4, 1, 5, 2, 5, 2, 5, 3, 2, 1, 2, 1, 3, 1, 5, 4, 1, 6, 4, 6, 4, 1, 5, 4, 
                  6, 3, 1, 3, 4, 5, 1, 5, 1, 2, 1, 5, 3, 4, 4, 1, 6, 4, 4, 6, 1, 4, 1, 4, 5, 1, 5, 4, 2, 3, 
                  4, 4, 2, 5, 1, 5, 3, 6, 4, 2, 3, 6, 5, 1, 1, 1, 4, 2, 3, 2, 4, 3, 4, 6, 6, 4, 5, 4, 6, 4, 
                  5, 4, 2, 3, 6, 4, 4, 1, 5, 6, 1, 5, 6, 1, 1, 3, 2, 3, 4, 5, 4, 5, 5, 2, 1, 2, 1, 4, 2, 3, 
                  6, 2, 4, 3, 5, 2, 6, 2, 1, 4, 5, 6, 3, 2, 6, 3, 3, 2, 6, 4, 1, 5, 4, 2, 5, 6, 3, 3, 2, 4, 
                  6, 4, 2, 5, 2, 6, 4, 2, 5, 2, 3, 4, 6, 6, 5, 5, 4, 1, 3, 5, 4, 3, 6, 4, 3, 4, 5, 4, 2, 6, 
                  6, 1, 3, 4, 4, 1, 4, 1, 1, 4, 1, 6, 4, 4, 1, 5, 1, 4, 1, 6, 1, 6, 3, 2, 2, 3, 4, 4, 6, 3, 
                  2, 3, 4, 1, 5, 5, 3, 1, 4, 5, 2, 4, 1, 1, 4, 5, 5, 3, 2, 3, 5, 5, 5, 3, 1, 4, 1, 2, 2, 5, 
                  4, 6, 1, 1, 4, 4, 6, 4, 5, 5, 1, 6, 4, 6, 5, 5]
    
    result_cpy = [0, 0, 0, 0, 0, 0, 0, 0]
    
    result_cpy[0] = result1[0] & result2[0] & result3[0] & result4[0] & result5[0] & result6[0] & result7[0] & result8[0] & result9[0] & result10[0] & result11[0] & result12[0] & result13[0] & result14[0] & result15[0] & result16[0] & result17[0] & result18[0] & result19[0] & result20[0] & result23[0] & result25[0] & result26[0] & result28[0] & result29[0] & result31[0] & result32[0] & result33[0] & result34[0] & result35[0] & result36[0] & result37[0] & result39[0] & result40[0] & result42[0] & result43[0] & result45[0] & result46[0] & result48[0] & result49[0] & result50[0] & result51[0] & result55[0] & result56[0] & result57[0] & result58[0] & result59[0] & result60[0] & result61[0] & result62[0] & result65[0] & result66[0] & result67[0] & result68[0] & result69[0] & result70[0] & result71[0] & result72[0] & result73[0] & result74[0] & result75[0] & result76[0] & result77[0] & result78[0] & result79[0] & result81[0] & result82[0] & result83[0] & result84[0] & result85[0] & result87[0] & result89[0] & result90[0] & result91[0] & result92[0] & result95[0] & result96[0] & result97[0] & result98[0] & result99[0] & result100[0] & result101[0] & result102[0] & result103[0] & result104[0] & result105[0] & result106[0] & result108[0] & result109[0] & result111[0] & result112[0] & result113[0] & result114[0] & result115[0] & result116[0] & result117[0] & result118[0] & result119[0] & result120[0] & result121[0] & result122[0] & result124[0] & result125[0] & result126[0] & result128[0]

    result_cpy[1] = result1[1] & result2[1] & result3[1] & result4[1] & result5[1] & result6[1] & result7[1] & result8[1] & result9[1] & result10[1] & result11[1] & result12[1] & result13[1] & result14[1] & result15[1] & result16[1] & result17[1] & result18[1] & result19[1] & result20[1] & result23[1] & result25[1] & result26[1] & result28[1] & result29[1] & result31[1] & result32[1] & result33[1] & result34[1] & result35[1] & result36[1] & result37[1] & result39[1] & result40[1] & result42[1] & result43[1] & result45[1] & result46[1] & result48[1] & result49[1] & result50[1] & result51[1] & result55[1] & result56[1] & result57[1] & result58[1] & result59[1] & result60[1] & result61[1] & result62[1] & result65[1] & result66[1] & result67[1] & result68[1] & result69[1] & result70[1] & result71[1] & result72[1] & result73[1] & result74[1] & result75[1] & result76[1] & result77[1] & result78[1] & result79[1] & result81[1] & result82[1] & result83[1] & result84[1] & result85[1] & result87[1] & result89[1] & result90[1] & result91[1] & result92[1] & result95[1] & result96[1] & result97[1] & result98[1] & result99[1] & result100[1] & result101[1] & result102[1] & result103[1] & result104[1] & result105[1] & result106[1] & result108[1] & result109[1] & result111[1] & result112[1] & result113[1] & result114[1] & result115[1] & result116[1] & result117[1] & result118[1] & result119[1] & result120[1] & result121[1] & result122[1] & result124[1] & result125[1] & result126[1] & result128[1]

    result_cpy[2] = result1[2] & result2[2] & result3[2] & result4[2] & result5[2] & result6[2] & result7[2] & result8[2] & result9[2] & result10[2] & result11[2] & result12[2] & result13[2] & result14[2] & result15[2] & result16[2] & result17[2] & result18[2] & result19[2] & result20[2] & result23[2] & result25[2] & result26[2] & result28[2] & result29[2] & result31[2] & result32[2] & result33[2] & result34[2] & result35[2] & result36[2] & result37[2] & result39[2] & result40[2] & result42[2] & result43[2] & result45[2] & result46[2] & result48[2] & result49[2] & result50[2] & result51[2] & result55[2] & result56[2] & result57[2] & result58[2] & result59[2] & result60[2] & result61[2] & result62[2] & result65[2] & result66[2] & result67[2] & result68[2] & result69[2] & result70[2] & result71[2] & result72[2] & result73[2] & result74[2] & result75[2] & result76[2] & result77[2] & result78[2] & result79[2] & result81[2] & result82[2] & result83[2] & result84[2] & result85[2] & result87[2] & result89[2] & result90[2] & result91[2] & result92[2] & result95[2] & result96[2] & result97[2] & result98[2] & result99[2] & result100[2] & result101[2] & result102[2] & result103[2] & result104[2] & result105[2] & result106[2] & result108[2] & result109[2] & result111[2] & result112[2] & result113[2] & result114[2] & result115[2] & result116[2] & result117[2] & result118[2] & result119[2] & result120[2] & result121[2] & result122[2] & result124[2] & result125[2] & result126[2] & result128[2]

    result_cpy[3] = result1[3] & result2[3] & result3[3] & result4[3] & result5[3] & result6[3] & result7[3] & result8[3] & result9[3] & result10[3] & result11[3] & result12[3] & result13[3] & result14[3] & result15[3] & result16[3] & result17[3] & result18[3] & result19[3] & result20[3] & result23[3] & result25[3] & result26[3] & result28[3] & result29[3] & result31[3] & result32[3] & result33[3] & result34[3] & result35[3] & result36[3] & result37[3] & result39[3] & result40[3] & result42[3] & result43[3] & result45[3] & result46[3] & result48[3] & result49[3] & result50[3] & result51[3] & result55[3] & result56[3] & result57[3] & result58[3] & result59[3] & result60[3] & result61[3] & result62[3] & result65[3] & result66[3] & result67[3] & result68[3] & result69[3] & result70[3] & result71[3] & result72[3] & result73[3] & result74[3] & result75[3] & result76[3] & result77[3] & result78[3] & result79[3] & result81[3] & result82[3] & result83[3] & result84[3] & result85[3] & result87[3] & result89[3] & result90[3] & result91[3] & result92[3] & result95[3] & result96[3] & result97[3] & result98[3] & result99[3] & result100[3] & result101[3] & result102[3] & result103[3] & result104[3] & result105[3] & result106[3] & result108[3] & result109[3] & result111[3] & result112[3] & result113[3] & result114[3] & result115[3] & result116[3] & result117[3] & result118[3] & result119[3] & result120[3] & result121[3] & result122[3] & result124[3] & result125[3] & result126[3] & result128[3]

    result_cpy[4] = result1[4] & result2[4] & result3[4] & result4[4] & result5[4] & result6[4] & result7[4] & result8[4] & result9[4] & result10[4] & result11[4] & result12[4] & result13[4] & result14[4] & result15[4] & result16[4] & result17[4] & result18[4] & result19[4] & result20[4] & result23[4] & result25[4] & result26[4] & result28[4] & result29[4] & result31[4] & result32[4] & result33[4] & result34[4] & result35[4] & result36[4] & result37[4] & result39[4] & result40[4] & result42[4] & result43[4] & result45[4] & result46[4] & result48[4] & result49[4] & result50[4] & result51[4] & result55[4] & result56[4] & result57[4] & result58[4] & result59[4] & result60[4] & result61[4] & result62[4] & result65[4] & result66[4] & result67[4] & result68[4] & result69[4] & result70[4] & result71[4] & result72[4] & result73[4] & result74[4] & result75[4] & result76[4] & result77[4] & result78[4] & result79[4] & result81[4] & result82[4] & result83[4] & result84[4] & result85[4] & result87[4] & result89[4] & result90[4] & result91[4] & result92[4] & result95[4] & result96[4] & result97[4] & result98[4] & result99[4] & result100[4] & result101[4] & result102[4] & result103[4] & result104[4] & result105[4] & result106[4] & result108[4] & result109[4] & result111[4] & result112[4] & result113[4] & result114[4] & result115[4] & result116[4] & result117[4] & result118[4] & result119[4] & result120[4] & result121[4] & result122[4] & result124[4] & result125[4] & result126[4] & result128[4]

    result_cpy[5] = result1[5] & result2[5] & result3[5] & result4[5] & result5[5] & result6[5] & result7[5] & result8[5] & result9[5] & result10[5] & result11[5] & result12[5] & result13[5] & result14[5] & result15[5] & result16[5] & result17[5] & result18[5] & result19[5] & result20[5] & result23[5] & result25[5] & result26[5] & result28[5] & result29[5] & result31[5] & result32[5] & result33[5] & result34[5] & result35[5] & result36[5] & result37[5] & result39[5] & result40[5] & result42[5] & result43[5] & result45[5] & result46[5] & result48[5] & result49[5] & result50[5] & result51[5] & result55[5] & result56[5] & result57[5] & result58[5] & result59[5] & result60[5] & result61[5] & result62[5] & result65[5] & result66[5] & result67[5] & result68[5] & result69[5] & result70[5] & result71[5] & result72[5] & result73[5] & result74[5] & result75[5] & result76[5] & result77[5] & result78[5] & result79[5] & result81[5] & result82[5] & result83[5] & result84[5] & result85[5] & result87[5] & result89[5] & result90[5] & result91[5] & result92[5] & result95[5] & result96[5] & result97[5] & result98[5] & result99[5] & result100[5] & result101[5] & result102[5] & result103[5] & result104[5] & result105[5] & result106[5] & result108[5] & result109[5] & result111[5] & result112[5] & result113[5] & result114[5] & result115[5] & result116[5] & result117[5] & result118[5] & result119[5] & result120[5] & result121[5] & result122[5] & result124[5] & result125[5] & result126[5] & result128[5]

    result_cpy[6] = result1[6] & result2[6] & result3[6] & result4[6] & result5[6] & result6[6] & result7[6] & result8[6] & result9[6] & result10[6] & result11[6] & result12[6] & result13[6] & result14[6] & result15[6] & result16[6] & result17[6] & result18[6] & result19[6] & result20[6] & result23[6] & result25[6] & result26[6] & result28[6] & result29[6] & result31[6] & result32[6] & result33[6] & result34[6] & result35[6] & result36[6] & result37[6] & result39[6] & result40[6] & result42[6] & result43[6] & result45[6] & result46[6] & result48[6] & result49[6] & result50[6] & result51[6] & result55[6] & result56[6] & result57[6] & result58[6] & result59[6] & result60[6] & result61[6] & result62[6] & result65[6] & result66[6] & result67[6] & result68[6] & result69[6] & result70[6] & result71[6] & result72[6] & result73[6] & result74[6] & result75[6] & result76[6] & result77[6] & result78[6] & result79[6] & result81[6] & result82[6] & result83[6] & result84[6] & result85[6] & result87[6] & result89[6] & result90[6] & result91[6] & result92[6] & result95[6] & result96[6] & result97[6] & result98[6] & result99[6] & result100[6] & result101[6] & result102[6] & result103[6] & result104[6] & result105[6] & result106[6] & result108[6] & result109[6] & result111[6] & result112[6] & result113[6] & result114[6] & result115[6] & result116[6] & result117[6] & result118[6] & result119[6] & result120[6] & result121[6] & result122[6] & result124[6] & result125[6] & result126[6] & result128[6]

    result_cpy[7] = result1[7] & result2[7] & result3[7] & result4[7] & result5[7] & result6[7] & result7[7] & result8[7] & result9[7] & result10[7] & result11[7] & result12[7] & result13[7] & result14[7] & result15[7] & result16[7] & result17[7] & result18[7] & result19[7] & result20[7] & result23[7] & result25[7] & result26[7] & result28[7] & result29[7] & result31[7] & result32[7] & result33[7] & result34[7] & result35[7] & result36[7] & result37[7] & result39[7] & result40[7] & result42[7] & result43[7] & result45[7] & result46[7] & result48[7] & result49[7] & result50[7] & result51[7] & result55[7] & result56[7] & result57[7] & result58[7] & result59[7] & result60[7] & result61[7] & result62[7] & result65[7] & result66[7] & result67[7] & result68[7] & result69[7] & result70[7] & result71[7] & result72[7] & result73[7] & result74[7] & result75[7] & result76[7] & result77[7] & result78[7] & result79[7] & result81[7] & result82[7] & result83[7] & result84[7] & result85[7] & result87[7] & result89[7] & result90[7] & result91[7] & result92[7] & result95[7] & result96[7] & result97[7] & result98[7] & result99[7] & result100[7] & result101[7] & result102[7] & result103[7] & result104[7] & result105[7] & result106[7] & result108[7] & result109[7] & result111[7] & result112[7] & result113[7] & result114[7] & result115[7] & result116[7] & result117[7] & result118[7] & result119[7] & result120[7] & result121[7] & result122[7] & result124[7] & result125[7] & result126[7] & result128[7]
    
    
    add_on = 0       
    if (result_cpy[7] != 0):
        add_on = 0
        result_cpy[0] = result_cpy[7]        
    elif (result_cpy[6] != 0):
        add_on = 32
        result_cpy[0] = result_cpy[6]
    elif (result_cpy[5] != 0):
        add_on = 64
        result_cpy[0] = result_cpy[5]        
    elif (result_cpy[4] != 0):
        add_on = 96
        result_cpy[0] = result_cpy[4]        
    elif (result_cpy[3] != 0):
        add_on = 128
        result_cpy[0] = result_cpy[3]
    elif (result_cpy[2] != 0):
        add_on = 160
        result_cpy[0] = result_cpy[2]        
    elif (result_cpy[1] != 0):
        add_on = 192
        result_cpy[0] = result_cpy[1]
    else:
        add_on = 224
        result_cpy[0] = result_cpy[0]    
    
    index = 0
    result_cpy[0] >>= 1
    while(result_cpy[0] != 0):
        result_cpy[0] >>= 1
        index += 1
        
    return path_class[index+add_on]
    
    

In [ ]:
clf_preds = clf.predict(X_test)
net_preds = []

In [ ]:
for i in range(len(X_test)):
    if_ex = [num_translate(feat, total_bits, dec_bits) for feat in X_test[i]]    
    
    result1 = feature1(if_ex[0])
    result2 = feature2(if_ex[1])
    result3 = feature3(if_ex[2])
    result4 = feature4(if_ex[3])
    result5 = feature5(if_ex[4])
    result6 = feature6(if_ex[5])
    result7 = feature7(if_ex[6])
    result8 = feature8(if_ex[7])
    result9 = feature9(if_ex[8])
    result10 = feature10(if_ex[9])
    result11 = feature11(if_ex[10])
    result12 = feature12(if_ex[11])
    result13 = feature13(if_ex[12])
    result14 = feature14(if_ex[13])
    result15 = feature15(if_ex[14])
    result16 = feature16(if_ex[15])
    result17 = feature17(if_ex[16])
    result18 = feature18(if_ex[17])
    result19 = feature19(if_ex[18])
    result20 = feature20(if_ex[19])
    result23 = feature23(if_ex[22])
    result25 = feature25(if_ex[24])
    result26 = feature26(if_ex[25])
    result28 = feature28(if_ex[27])
    result29 = feature29(if_ex[28])
    result31 = feature31(if_ex[30])
    result32 = feature32(if_ex[31])
    result33 = feature33(if_ex[32])
    result34 = feature34(if_ex[33])
    result35 = feature35(if_ex[34])
    result36 = feature36(if_ex[35])
    result37 = feature37(if_ex[36])
    result39 = feature39(if_ex[38])
    result40 = feature40(if_ex[39])
    result42 = feature42(if_ex[41])
    result43 = feature43(if_ex[42])
    result45 = feature45(if_ex[44])
    result46 = feature46(if_ex[45])
    result48 = feature48(if_ex[47])
    result49 = feature49(if_ex[48])
    result50 = feature50(if_ex[49])
    result51 = feature51(if_ex[50])
    result55 = feature55(if_ex[54])
    result56 = feature56(if_ex[55])
    result57 = feature57(if_ex[56])
    result58 = feature58(if_ex[57])
    result59 = feature59(if_ex[58])
    result60 = feature60(if_ex[59])
    result61 = feature61(if_ex[60])
    result62 = feature62(if_ex[61])
    result65 = feature65(if_ex[64])
    result66 = feature66(if_ex[65])
    result67 = feature67(if_ex[66])
    result68 = feature68(if_ex[67])
    result69 = feature69(if_ex[68])
    result70 = feature70(if_ex[69])
    result71 = feature71(if_ex[70])
    result72 = feature72(if_ex[71])
    result73 = feature73(if_ex[72])
    result74 = feature74(if_ex[73])
    result75 = feature75(if_ex[74])
    result76 = feature76(if_ex[75])
    result77 = feature77(if_ex[76])
    result78 = feature78(if_ex[77])
    result79 = feature79(if_ex[78])
    result81 = feature81(if_ex[80])
    result82 = feature82(if_ex[81])
    result83 = feature83(if_ex[82])
    result84 = feature84(if_ex[83])
    result85 = feature85(if_ex[84])
    result87 = feature87(if_ex[86])
    result89 = feature89(if_ex[88])
    result90 = feature90(if_ex[89])
    result91 = feature91(if_ex[90])
    result92 = feature92(if_ex[91])
    result95 = feature95(if_ex[94])
    result96 = feature96(if_ex[95])
    result97 = feature97(if_ex[96])
    result98 = feature98(if_ex[97])
    result99 = feature99(if_ex[98])
    result100 = feature100(if_ex[99])
    result101 = feature101(if_ex[100])
    result102 = feature102(if_ex[101])
    result103 = feature103(if_ex[102])
    result104 = feature104(if_ex[103])
    result105 = feature105(if_ex[104])
    result106 = feature106(if_ex[105])
    result108 = feature108(if_ex[107])
    result109 = feature109(if_ex[108])
    result111 = feature111(if_ex[110])
    result112 = feature112(if_ex[111])
    result113 = feature113(if_ex[112])
    result114 = feature114(if_ex[113])
    result115 = feature115(if_ex[114])
    result116 = feature116(if_ex[115])
    result117 = feature117(if_ex[116])
    result118 = feature118(if_ex[117])
    result119 = feature119(if_ex[118])
    result120 = feature120(if_ex[119])
    result121 = feature121(if_ex[120])
    result122 = feature122(if_ex[121])
    result124 = feature124(if_ex[123])
    result125 = feature125(if_ex[124])
    result126 = feature126(if_ex[125])
    result128 = feature128(if_ex[127])
    
    if_pred = dtree_aggr(result1, result2, result3, result4, result5, result6, result7, result8, result9, 
                         result10, result11, result12, result13, result14, result15, result16, result17, 
                         result18, result19, result20, result23, result25, result26, result28, result29, 
                         result31, result32, result33, result34, result35, result36, result37, result39,
                         result40, result42, result43, result45, result46, result48, result49, result50, 
                         result51, result55, result56, result57, result58, result59, result60, result61,  
                         result62, result65, result66, result67, result68, result69, result70, result71, 
                         result72, result73, result74, result75, result76, result77, result78, result79,
                         result81, result82, result83, result84, result85, result87, result89, result90, 
                         result91, result92, result95, result96, result97, result98, result99, result100, 
                         result101, result102, result103, result104, result105, result106, result108, 
                         result109, result111, result112, result113, result114, result115, result116, 
                         result117, result118, result119, result120, result121, result122, result124, 
                         result125, result126, result128)
    
    net_preds.append(if_pred)

In [ ]:
y_test = y_test.to_numpy()

In [ ]:
print(accuracy_score(y_test, net_preds) * 100)
print(f1_score(y_test, net_preds, average='macro') * 100)
print(precision_score(y_test, net_preds, average='macro') * 100)
print(recall_score(y_test, net_preds, average='macro') * 100)